In [4]:
import pandas as pd
social = pd.read_csv("Aggregated_reddit_twitter.csv")
crypto = pd.read_csv("transformed_crypto.csv")

C:\Users\Dario\AppData\Local\Temp\ipykernel_45676\1289106569.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  social = pd.read_csv("Aggregated_reddit_twitter.csv")


In [7]:
social["date"] = pd.to_datetime(social["date"])
social.describe()

,Unnamed: 0,replies,likes,other_scores
count,1.630626e+06,1.608590e+06,1.608590e+06,0.0
mean,8.153125e+05,1.487198e+00,9.932480e+00,NaN
std,4.707213e+05,3.940739e+01,1.757894e+02,NaN
min,0.000000e+00,0.000000e+00,-2.086000e+03,NaN
25%,4.076562e+05,0.000000e+00,1.000000e+00,NaN
50%,8.153125e+05,0.000000e+00,1.000000e+00,NaN
75%,1.222969e+06,0.000000e+00,3.000000e+00,NaN
max,1.630625e+06,1.172700e+04,6.685200e+04,NaN
